In [67]:
!npm install dukascopy-node --save


added 30 packages, and audited 31 packages in 7s

1 package is looking for funding
  run `npm fund` for details

found 0 vulnerabilities


npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm notice 
npm notice New major version of npm available! 8.11.0 -> 9.6.5
npm notice Changelog: <https://github.com/npm/cli/releases/tag/v9.6.5>
npm notice Run `npm install -g npm@9.6.5` to update!
npm notice 


In [107]:
!npx dukascopy-node -i usdcad -from 2023-04-21 -to 2023-04-23 -t h1 -f json --volumes


----------------------------------------------------
----------------------------------------------------
Instrument:     US Dollar vs Canadian Dollar
Timeframe:      h1
From date:      Apr 21, 2023, 12 AM
To date:        Apr 21, 2023, 9 PM
Price type:     bid
Volumes:        true
UTC Offset:     0
Include flats:  false
Format:         json
----------------------------------------------------
----------------------------------------------------
âˆš File saved: download\usdcad-h1-bid-2023-04-21-2023-04-21.json (212 Bytes)

Download time: 6.1s



npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [106]:
!npx dukascopy-node -i brentcmdusd -from 2023-04-21 -to 2023-04-23 -t h1 -f json --volumes


----------------------------------------------------
----------------------------------------------------
Instrument:     US Brent Crude Oil
Timeframe:      h1
From date:      Apr 21, 2023, 12 AM
To date:        Apr 21, 2023, 9 PM
Price type:     bid
Volumes:        true
UTC Offset:     0
Include flats:  false
Format:         json
----------------------------------------------------
----------------------------------------------------
âˆš File saved: download\brentcmdusd-h1-bid-2023-04-21-2023-04-21.json (405 Bytes)

Download time: 6.8s



npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [114]:
import json
import pandas as pd

# abre e carrega o conteúdo do arquivo JSON do par USDCAD
with open('download/usdcad-h1-bid-2023-04-21-2023-04-21.json') as f:
    data_usdcad = json.load(f)

# seleciona o último item da lista
ultimo_item_usdcad = data_usdcad[-1]

# abre e carrega o conteúdo do arquivo JSON do Brent crude oil
with open('download/brentcmdusd-h1-bid-2023-04-21-2023-04-21.json') as f:
    data_oil = json.load(f)

# seleciona o último item da lista
ultimo_item_oil = data_oil[-1]

# cria um dicionário com as informações dos últimos itens
ultimo_item_dict = {
    'Open_usdcad': ultimo_item_usdcad['open'],
    'High_usdcad': ultimo_item_usdcad['high'],
    'Low_usdcad': ultimo_item_usdcad['low'],
    'Close_usdcad': ultimo_item_usdcad['close'],
    'Volume_usdcad': ultimo_item_usdcad['volume'],
    'Open_oil': ultimo_item_oil['open'],
    'High_oil': ultimo_item_oil['high'],
    'Low_oil': ultimo_item_oil['low'],
    'Close_oil': ultimo_item_oil['close'],
    'Volume_oil': ultimo_item_oil['volume']
}

# converte o dicionário em um dataframe
df = pd.DataFrame.from_dict([ultimo_item_dict])

# adiciona as colunas "year", "month", "day" e "hour"
df['year'] = pd.to_datetime(ultimo_item_usdcad['timestamp'], unit='ms').year
df['month'] = pd.to_datetime(ultimo_item_usdcad['timestamp'], unit='ms').month
df['day'] = pd.to_datetime(ultimo_item_usdcad['timestamp'], unit='ms').day
df['hour'] = pd.to_datetime(ultimo_item_usdcad['timestamp'], unit='ms').hour

# exibe o dataframe
df


,Open_usdcad,High_usdcad,Low_usdcad,Close_usdcad,Volume_usdcad,Open_oil,High_oil,Low_oil,Close_oil,Volume_oil,year,month,day,hour
0,1.35375,1.3541,1.35338,1.3534,1892.87,81.565,81.785,81.525,81.745,0.0157,2023,4,21,20
